In [1]:
import pytz

from metalib.metagomano import MetaGO
from MetaTrader5 import TIMEFRAME_M1
from datetime import datetime, timedelta
import pytz as pytz

In [2]:
end_time = datetime.now(pytz.utc) + timedelta(hours=3)
start_time = end_time - timedelta(days=30)

In [3]:
from MetaTrader5 import copy_rates_range
import MetaTrader5 as mt

In [4]:
import yaml
import warnings
from multiprocessing import Process
from metalib.metaworker import run_strategy_loop
import MetaTrader5 as mt5

warnings.filterwarnings("ignore")

processes = []

with open("../config/dev/metago.yaml", "r") as f:
    config_data = yaml.safe_load(f)

name = "BTCUSD_H1_GO"
entry = config_data[name]


In [5]:
strategy_type = entry.pop("strategy_type")
init_args = entry.copy()

# Convert timeframe string (e.g. "TIMEFRAME_M1") to actual mt5 constant
if isinstance(init_args.get("timeframe"), str):
    init_args["timeframe"] = eval(init_args["timeframe"])

# Convert null active_hours to None
if "active_hours" in init_args and init_args["active_hours"] == "None":
    init_args["active_hours"] = None

In [6]:
strategy_registry = {
    "metago": {
        "class": MetaGO,
        "init_keys": [
            "symbols",
            "timeframe",
            "tag",
            "active_hours",
        ]
    }
}

In [7]:
strategy_registry

{'metago': {'class': metalib.metagomano.MetaGO,
  'init_keys': ['symbols', 'timeframe', 'tag', 'active_hours']}}

In [8]:
init_args

{'symbols': ['BTCUSD'],
 'timeframe': 1,
 'tag': 'metago_crypto_btc',
 'active_hours': None,
 'risk_factor': 0.09,
 'lookahead': 48}

In [9]:
strategy_info = strategy_registry.get(strategy_type)
if not strategy_info:
    raise ValueError(f"Unknown strategy type: {strategy_type}")

StrategyClass = strategy_info["class"]
expected_keys = strategy_info["init_keys"]

# Ensure required keys are present
for key in expected_keys:
    if key not in init_args:
        raise ValueError(f"Missing required argument: {key}")

instance = StrategyClass(**{k: init_args[k] for k in expected_keys})
instance.connect()
instance.fit()

No model to fit for MetaGO(mano) pelo!


In [10]:
instance.run(start_time, end_time)

True

In [11]:
instance.loadData(start_time, end_time)

In [12]:
instance.sl

115070.57857142859

In [13]:
instance.tp

118050.22

In [15]:
type(instance.signalData)

pandas.core.frame.DataFrame

In [11]:
instance.run(start_time, end_time)

True